# Multi-Agent Service Matcher

## Overview

This notebook implements a multi-agent system that matches user requests to services from a catalog. The system uses three specialized LLM-powered agents in a sequential pipeline:

1. **Interpreter Agent** - Normalizes and clarifies user input
2. **Matcher Agent** - Identifies the best matching service using natural language reasoning
3. **Polisher Agent** - Formats the result into a user-friendly response

The implementation demonstrates:
- Sequential multi-agent pipeline architecture
- Custom tool integration for data access
- In-memory session state management
- Natural language reasoning for semantic matching

---

## System Architecture

```
User Request
     |
     v
┌─────────────────────────┐
│  AGENT 1: INTERPRETER   │  Role: Clarify and normalize the user's request
│  Input: Raw user text   │
│  Output: Clear summary  │
└──────────┬──────────────┘
           │ (writes to session)
           v
┌─────────────────────────┐
│  AGENT 2: MATCHER       │  Role: Find the best matching service
│  Input: Summary + Tool  │  Tool: get_services() → returns service list
│  Output: Best match     │
└──────────┬──────────────┘
           │ (writes to session)
           v
┌─────────────────────────┐
│  AGENT 3: POLISHER      │  Role: Format user-friendly final answer
│  Input: Matched service │
│  Output: Final response │
└──────────┬──────────────┘
           │
           v
    Final Answer
```

---

## Environment Setup

The system uses Google Vertex AI for LLM calls. Vertex AI provides enterprise-grade access to Gemini models through Google Cloud Platform.

In [ ]:
# Dependencies are managed via requirements.txt
# Install them once in your terminal before running this notebook:
#   pip install -r requirements.txt
#
# If running in Google Colab, uncomment the line below:
# !pip install -q google-cloud-aiplatform python-dotenv

In [ ]:
import json
import os
from typing import Dict, List, Any
from dotenv import load_dotenv
from google import genai

# Load environment variables from .env file
load_dotenv()

# Get configuration from environment variables
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
LOCATION = os.getenv("GOOGLE_CLOUD_LOCATION", "us-central1")

print("="*60)
print("🔧 Google GenAI Configuration")
print("="*60)
print(f"Project ID: {PROJECT_ID}")
print(f"Location: {LOCATION}")
print()

try:
    # Initialize Google GenAI client with Vertex AI
    client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
    
    # Test with a simple query to verify it works
    print("Testing connection...")
    test_response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents="Say 'Ready!' if you can read this"
    )
    print(f"✅ Google GenAI initialized successfully!")
    print(f"✅ Model response: {test_response.text.strip()}")
    print()
    print("Ready to use Gemini models via Vertex AI")
except Exception as e:
    print("❌ Failed to initialize Google GenAI")
    print(f"Error: {e}")
    print()
    print("Please ensure:")
    print("  1. You've run: gcloud auth application-default login")
    print("  2. Your project has Vertex AI API enabled")
    print("  3. You have the correct permissions")

🔧 Google GenAI Configuration
Project ID: d-ulti-ml-ds-dev-9561
Location: us-central1

Testing connection...
✅ Google GenAI initialized successfully!
✅ Model response: Ready!

Ready to use Gemini models via Vertex AI


---

## Service Database Tool

The `get_services()` function provides access to the service catalog. In production, this would query a database or API.

In [ ]:
def get_services() -> List[Dict[str, Any]]:
    """
    Returns the list of available services.
    
    Returns:
        List of service dictionaries, each containing:
        - service_id: Unique identifier
        - service_title: Name of the service
        - service_description: What the service does
    """
    
    services = [
        {
            "service_id": 1,
            "service_title": "Python Debug Helper",
            "service_description": "I help fix Python bugs quickly using simple explanations."
        },
        {
            "service_id": 2,
            "service_title": "Tax Filing Advisor",
            "service_description": "I answer questions about government tax forms and common filing issues."
        },
        {
            "service_id": 3,
            "service_title": "Swimming Technique Review",
            "service_description": "I give fast feedback on stroke mechanics, breathing, and body position."
        },
        {
            "service_id": 4,
            "service_title": "SAT Math Tutor",
            "service_description": "I explain SAT math problems step-by-step and help improve accuracy."
        }
    ]
    
    return services


# Test the tool
print("🔧 Testing get_services() tool:\n")
services = get_services()
for service in services:
    print(f"  [{service['service_id']}] {service['service_title']}")
    print(f"      → {service['service_description']}")
    print()

print(f"✓ Tool returns {len(services)} services")

🔧 Testing get_services() tool:

  [1] Python Debug Helper
      → I help fix Python bugs quickly using simple explanations.

  [2] Tax Filing Advisor
      → I answer questions about government tax forms and common filing issues.

  [3] Swimming Technique Review
      → I give fast feedback on stroke mechanics, breathing, and body position.

  [4] SAT Math Tutor
      → I explain SAT math problems step-by-step and help improve accuracy.

✓ Tool returns 4 services


---

## Session State Implementation

Session state is a shared dictionary that agents use to pass information through the pipeline. Each agent reads from and writes to specific keys in the session.

In [ ]:
def create_session(user_request: str) -> Dict[str, Any]:
    """
    Initialize a new session with the user's request.
    
    Args:
        user_request: The raw text from the user
        
    Returns:
        A new session dictionary
    """
    return {
        "user_request": user_request,
        "clarified_request": None,
        "matched_service": None,
        "final_response": None
    }


def print_session_state(session: Dict[str, Any], title: str = "Session State"):
    """
    Display the current session state.
    """
    print(f"\n{'='*60}")
    print(f"  {title}")
    print(f"{'='*60}")
    for key, value in session.items():
        print(f"\n{key}:")
        if isinstance(value, dict):
            print(json.dumps(value, indent=2))
        else:
            print(f"  {value}")
    print(f"\n{'='*60}\n")


# Test session creation
test_session = create_session("I need help with my Python code")
print_session_state(test_session, "Example: Fresh Session")


  Example: Fresh Session

user_request:
  I need help with my Python code

clarified_request:
  None

matched_service:
  None

final_response:
  None




---

## Agent 1: Interpreter

The Interpreter agent normalizes user input into a clear, structured summary.

In [ ]:
def agent_1_interpreter(session: Dict[str, Any]) -> Dict[str, Any]:
    """
    AGENT 1: INTERPRETER
    
    Clarifies and normalizes the user's raw request into a clean summary.
    
    Input (from session):
        - user_request: Raw text from the user
        
    Output (written to session):
        - clarified_request: A clear, normalized summary
        
    Returns:
        Updated session dictionary
    """
    
    print("\n🤖 AGENT 1: INTERPRETER is working...")
    
    user_request = session["user_request"]
    print(f"   Reading user request: '{user_request}'")
    
    prompt = f"""
You are an interpreter agent. Your job is to take a user's informal, possibly messy request and rewrite it into a clear, concise summary.

Rules:
1. Keep it to one or two sentences
2. Focus on the main need or problem
3. Remove filler words and casual language
4. Don't add information that wasn't in the original request
5. Don't try to solve the problem - just clarify what they're asking for

User request: "{user_request}"

Provide only the clarified summary, nothing else.
"""
    
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=prompt
    )
    clarified_request = response.text.strip()
    
    print(f"   ✓ Clarified to: '{clarified_request}'")
    
    session["clarified_request"] = clarified_request
    
    return session


# Test Agent 1
print("\n" + "="*60)
print("Testing Agent 1: Interpreter")
print("="*60)

test_session = create_session("my python loop keeps skipping stuff idk why")
test_session = agent_1_interpreter(test_session)

print("\nResult:")
print(f"  Original: {test_session['user_request']}")
print(f"  Clarified: {test_session['clarified_request']}")


Testing Agent 1: Interpreter

🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'my python loop keeps skipping stuff idk why'
   ✓ Clarified to: 'I need help understanding why my Python loop is skipping iterations.'

Result:
  Original: my python loop keeps skipping stuff idk why
  Clarified: I need help understanding why my Python loop is skipping iterations.


---

## Agent 2: Matcher

The Matcher agent identifies the best service by calling the `get_services()` tool and using LLM reasoning to compare the user's need against available services.

In [ ]:
def agent_2_matcher(session: Dict[str, Any]) -> Dict[str, Any]:
    """
    AGENT 2: MATCHER
    
    Finds the best matching service by:
    1. Calling the get_services() tool
    2. Using LLM reasoning to compare the user's need against each service
    3. Selecting the single best match
    4. Explaining the selection
    
    Input (from session):
        - clarified_request: The normalized user request from Agent 1
        
    Output (written to session):
        - matched_service: Dictionary containing service details and match reason
        
    Returns:
        Updated session dictionary
    """
    
    print("\n🤖 AGENT 2: MATCHER is working...")
    
    clarified_request = session["clarified_request"]
    print(f"   Reading clarified request: '{clarified_request}'")
    
    print("   Calling get_services() tool...")
    services = get_services()
    print(f"   ✓ Retrieved {len(services)} services")
    
    services_text = "\n".join([
        f"Service {s['service_id']}: {s['service_title']}\n  Description: {s['service_description']}"
        for s in services
    ])
    
    prompt = f"""
You are a matcher agent. Your job is to find the single best matching service for a user's request.

User's need: "{clarified_request}"

Available services:
{services_text}

Instructions:
1. Read the user's need carefully
2. Compare it against each service's title and description
3. Use reasoning to determine which service is the BEST match
4. Provide a short explanation of WHY this service matches best

Return your answer as JSON with this exact structure:
{{
  "service_id": <number>,
  "service_title": "<exact title>",
  "service_description": "<exact description>",
  "reason": "<your explanation of why this is the best match>"
}}

Return ONLY the JSON, no other text.
"""
    
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=prompt
    )
    
    # Extract JSON from response
    response_text = response.text.strip()
    if response_text.startswith("```"):
        response_text = response_text.split("```")[1]
        if response_text.startswith("json"):
            response_text = response_text[4:]
    matched_service = json.loads(response_text.strip())
    
    print(f"   ✓ Matched to: {matched_service['service_title']}")
    print(f"   Reason: {matched_service['reason']}")
    
    session["matched_service"] = matched_service
    
    return session


# Test Agent 2
print("\n" + "="*60)
print("Testing Agent 2: Matcher")
print("="*60)

test_session = create_session("I need help with Python loops")
test_session["clarified_request"] = "User needs help debugging Python for-loops"
test_session = agent_2_matcher(test_session)

print("\nResult:")
print(json.dumps(test_session["matched_service"], indent=2))


Testing Agent 2: Matcher

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'User needs help debugging Python for-loops'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Python Debug Helper
   Reason: The user needs help debugging Python code, specifically for-loops, and the Python Debug Helper service is designed to help fix Python bugs.

Result:
{
  "service_id": 1,
  "service_title": "Python Debug Helper",
  "service_description": "I help fix Python bugs quickly using simple explanations.",
  "reason": "The user needs help debugging Python code, specifically for-loops, and the Python Debug Helper service is designed to help fix Python bugs."
}


---

## Agent 3: Polisher

The Polisher agent formats the matched service into a user-friendly final response.

In [ ]:
def agent_3_polisher(session: Dict[str, Any]) -> Dict[str, Any]:
    """
    AGENT 3: POLISHER
    
    Creates a user-friendly final response from the matched service.
    
    Input (from session):
        - matched_service: The service selected by Agent 2, with reason
        
    Output (written to session):
        - final_response: A polished, conversational answer for the user
        
    Returns:
        Updated session dictionary
    """
    
    print("\n🤖 AGENT 3: POLISHER is working...")
    
    matched_service = session["matched_service"]
    print(f"   Reading matched service: {matched_service['service_title']}")
    
    prompt = f"""
You are a polisher agent. Your job is to create a friendly, clear final response for the user.

You have matched the user to this service:
- Title: {matched_service['service_title']}
- Description: {matched_service['service_description']}
- Reason for match: {matched_service['reason']}

Create a response with this structure:

Best Match: [service title]
Description: [service description]
Why: [explain why this service matches the user's need]

Keep it concise and friendly. The "Why" should be conversational and clear.
"""
    
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=prompt
    )
    final_response = response.text.strip()
    
    print("   ✓ Final response created")
    
    session["final_response"] = final_response
    
    return session


# Test Agent 3
print("\n" + "="*60)
print("Testing Agent 3: Polisher")
print("="*60)

test_session = create_session("test")
test_session["matched_service"] = {
    "service_id": 4,
    "service_title": "SAT Math Tutor",
    "service_description": "I explain SAT math problems step-by-step and help improve accuracy.",
    "reason": "The request mentions SAT math word problems, which directly aligns with this service."
}
test_session = agent_3_polisher(test_session)

print("\nResult:")
print(test_session["final_response"])


Testing Agent 3: Polisher

🤖 AGENT 3: POLISHER is working...
   Reading matched service: SAT Math Tutor
   ✓ Final response created

Result:
Best Match: SAT Math Tutor
Description: I explain SAT math problems step-by-step and help improve accuracy.
Why: I saw you mentioned SAT math word problems, and that's exactly what this tutor specializes in! They can help you break down those problems and boost your confidence.


---

## Pipeline Orchestration

The `run_pipeline()` function orchestrates the sequential execution of all three agents, passing session state through each stage.

In [ ]:
def run_pipeline(user_request: str, verbose: bool = True) -> str:
    """
    Orchestrates the multi-agent pipeline.
    
    Args:
        user_request: The raw text from the user
        verbose: If True, print detailed progress information
        
    Returns:
        The final formatted response string
    """
    
    if verbose:
        print("\n" + "="*60)
        print("🚀 STARTING MULTI-AGENT PIPELINE")
        print("="*60)
        print(f"\nUser Request: \"{user_request}\"\n")
    
    session = create_session(user_request)
    
    session = agent_1_interpreter(session)
    session = agent_2_matcher(session)
    session = agent_3_polisher(session)
    
    final_response = session["final_response"]
    
    if verbose:
        print("\n" + "="*60)
        print("✅ PIPELINE COMPLETE")
        print("="*60)
    
    return final_response


def match_service(user_request: str):
    """
    Convenience function that runs the pipeline and displays the result.
    """
    result = run_pipeline(user_request, verbose=True)
    print("\n" + "="*60)
    print("📋 FINAL RESPONSE")
    print("="*60)
    print("\n" + result + "\n")
    return result

---

## Usage Examples

### Example 1: Python Debugging Request

In [ ]:
match_service("my python code keeps crashing when i try to open a file help!!")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "my python code keeps crashing when i try to open a file help!!"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'my python code keeps crashing when i try to open a file help!!'
   ✓ Clarified to: 'My Python code crashes when attempting to open a file, and I need assistance.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'My Python code crashes when attempting to open a file, and I need assistance.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Python Debug Helper
   Reason: The user's problem is a crashing Python code, specifically related to file opening. The Python Debug Helper is designed to fix Python bugs, making it the most relevant service.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: Python Debug Helper
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Best Match: Python Debug Helper
Description: I help fix Python bugs quickly usi

"Best Match: Python Debug Helper\nDescription: I help fix Python bugs quickly using simple explanations.\nWhy: Your Python code is crashing when trying to open a file, which definitely sounds like a bug! The Python Debug Helper is here to help you squash that bug and get your code running smoothly. Let's figure out what's going wrong with that file opening!"

### Example 2: SAT Math Help

In [ ]:
match_service("my kid needs help with sat math word problems")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "my kid needs help with sat math word problems"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'my kid needs help with sat math word problems'
   ✓ Clarified to: 'My child requires assistance with SAT math word problems.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'My child requires assistance with SAT math word problems.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: SAT Math Tutor
   Reason: The user specifically needs help with SAT math word problems, and the SAT Math Tutor service directly addresses this need by offering step-by-step explanations and aiming to improve accuracy in SAT math.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: SAT Math Tutor
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Okay, here's the response I've created:

Best Match: SAT Math Tutor
Description: I explain SAT math problems step-by-step and help impro

"Okay, here's the response I've created:\n\nBest Match: SAT Math Tutor\nDescription: I explain SAT math problems step-by-step and help improve accuracy.\nWhy: This looks like a great fit! You mentioned needing help with SAT math word problems, and this tutor focuses specifically on that by breaking down problems and helping you get those answers right!"

### Example 3: Tax Questions

In [ ]:
match_service("i have questions about filing my taxes this year")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "i have questions about filing my taxes this year"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'i have questions about filing my taxes this year'
   ✓ Clarified to: 'Clarify questions about this year's tax filing process.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'Clarify questions about this year's tax filing process.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Tax Filing Advisor
   Reason: The user needs help with tax filing questions, and the Tax Filing Advisor service specifically addresses questions about tax forms and filing issues.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: Tax Filing Advisor
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Okay, here's the polished response:

Best Match: Tax Filing Advisor
Description: I answer questions about government tax forms and common filing issues.
Why: It sounds like you're 

"Okay, here's the polished response:\n\nBest Match: Tax Filing Advisor\nDescription: I answer questions about government tax forms and common filing issues.\nWhy: It sounds like you're looking for help with tax filing, and the Tax Filing Advisor is designed to answer questions about tax forms and any filing issues you might be running into! Let me know what's on your mind!"

### Example 4: Swimming Technique

In [ ]:
match_service("I want to improve my freestyle stroke breathing")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "I want to improve my freestyle stroke breathing"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'I want to improve my freestyle stroke breathing'
   ✓ Clarified to: 'Clarify techniques for freestyle stroke breathing.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'Clarify techniques for freestyle stroke breathing.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Swimming Technique Review
   Reason: The user's request is about freestyle stroke breathing techniques, which directly aligns with the description of the Swimming Technique Review service.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: Swimming Technique Review
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Okay, here's a response tailored for the user:

Best Match: Swimming Technique Review
Description: I give fast feedback on stroke mechanics, breathing, and body position.
Why: 

"Okay, here's a response tailored for the user:\n\nBest Match: Swimming Technique Review\nDescription: I give fast feedback on stroke mechanics, breathing, and body position.\nWhy: This service sounds perfect for you! Since you're asking about freestyle stroke breathing techniques, I can give you some quick feedback on that, along with your overall stroke mechanics and body position. Let's get you swimming even better!"

### Example 5: Edge Case - Ambiguous Request

In [ ]:
match_service("I need help with numbers")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "I need help with numbers"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'I need help with numbers'
   ✓ Clarified to: 'The user requires assistance with numerical information.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'The user requires assistance with numerical information.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: SAT Math Tutor
   Reason: The user needs assistance with numerical information, and the SAT Math Tutor focuses specifically on explaining and improving accuracy with math problems, making it the best match among the available services.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: SAT Math Tutor
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Okay, here's the polished response for the user:

Best Match: SAT Math Tutor
Description: I explain SAT math problems step-by-step and help improve accuracy.
Why: You mention

"Okay, here's the polished response for the user:\n\nBest Match: SAT Math Tutor\nDescription: I explain SAT math problems step-by-step and help improve accuracy.\nWhy: You mentioned needing help with numerical information, and the SAT Math Tutor is all about explaining math problems clearly and helping you get more accurate answers. Seems like a great fit!"

---

## Session State Inspection

The pipeline can be run with full session state visibility for debugging purposes.

In [ ]:
def run_pipeline_with_inspection(user_request: str):
    """
    Run the pipeline with session state inspection after each agent.
    """
    print("\n" + "="*60)
    print("🔍 RUNNING PIPELINE WITH INSPECTION")
    print("="*60)
    
    session = create_session(user_request)
    print_session_state(session, "Initial State")
    
    session = agent_1_interpreter(session)
    print_session_state(session, "After Agent 1 (Interpreter)")
    
    session = agent_2_matcher(session)
    print_session_state(session, "After Agent 2 (Matcher)")
    
    session = agent_3_polisher(session)
    print_session_state(session, "After Agent 3 (Polisher)")
    
    return session["final_response"]

run_pipeline_with_inspection("python loop broken help")


🔍 RUNNING PIPELINE WITH INSPECTION

  Initial State

user_request:
  python loop broken help

clarified_request:
  None

matched_service:
  None

final_response:
  None



🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'python loop broken help'
   ✓ Clarified to: 'The user needs help debugging a broken loop in their Python code.'

  After Agent 1 (Interpreter)

user_request:
  python loop broken help

clarified_request:
  The user needs help debugging a broken loop in their Python code.

matched_service:
  None

final_response:
  None



🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'The user needs help debugging a broken loop in their Python code.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Python Debug Helper
   Reason: The user specifically needs help debugging Python code, and the Python Debug Helper is designed for that purpose.

  After Agent 2 (Matcher)

user_request:
  python loop broken help

clarified_request:

"Okay, here's the response:\n\nBest Match: Python Debug Helper\nDescription: I help fix Python bugs quickly using simple explanations.\nWhy: You're looking for help debugging your Python code, and that's exactly what I'm designed to do! I'll help you squash those bugs!"